In [ ]:
import os
import sys
import numpy
import math
import collections

from collections import Counter
import cryptography.hazmat
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import CTR # Might use, has parallel encryption
from cryptography.hazmat.primitives.ciphers.modes import CBC
from cryptography.hazmat.primitives.ciphers.modes import XTS # Only for AES, but this is disk encryption mode
from cryptography.hazmat.backends import default_backend

In [ ]:
# Fernet needs a key in constructor
# So lets build one from a key derivation scheme
# I'm using PBKDF2

# A few notes
# AES with 128 bit keys is fine
# So we only need 128/8 = 16 bytes of key
# We, however, will use AES256 because we can
# In the worst case, 256 performs like it has 131 bits of key

# Because we are using XTS, we need to double the key size to 512bit

kdf = PBKDF2HMAC(
    # Commented below is what would have been used for CBC
    algorithm=hashes.SHA512_256(), # Faster than 256 on 64bit machines
    length=32,
    salt=os.urandom(32),
    iterations=1000000,
    backend=default_backend()
)

xtskdf = PBKDF2HMAC(
    # Commented below is what would have been used for CBC
    #algorithm=hashes.SHA512_256(), # Faster than 256 on 64bit machines
    #length=32,
    algorithm=hashes.SHA512(),
    length=64,
    salt=os.urandom(32),
    iterations=1000000,
    backend=default_backend()
)


In [ ]:
# "Cache" in the notebook
# Otherwise we get an exception for using the KDF twice
#key = kdf.derive(b"Password")
key = kdf.derive(b"testPW")
#key = kdf.derive(os.urandom(16))
xtskey = xtskdf.derive(b"Password")

In [ ]:
# Construct an AES CBC Context

# Don't actually use that as the password!
algo = AES(key)

# We don't care about the IV
# We can use a dead block at the start
mode = CBC(os.urandom(16))

# Now we get our two functions
# We'll just use encryption for this but it's a good example
# NOTE: This can throw cryptography.exceptions.UnsupportedAlgorithm
cipher = Cipher(algo, mode, backend=default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
# XTS
def DoXTSSectorEncryption(key, secID):
    # We want to get the IV from disk sector if possible
    xtsalgo = AES(key)
    xtsmode = XTS(secID)
    



In [ ]:
def GetBlockDeviceSize(blk):
    sizeOfBlockDevice = 0
    try:
        sizeOfBlockDevice = os.lseek(blk, 0, os.SEEK_END)
    finally:
        os.lseek(blk, 0, os.SEEK_SET)
        
    return sizeOfBlockDevice

In [ ]:
def GetBlockDeviceSizeSafe(blk):
    rawBlkSize = GetBlockDeviceSize(blk)
    FullSafeSize = rawBlkSize // 16 # AES Block Size
    #sectors -= 1 # For Dead Block
    return FullSafeSize * 16
    

In [ ]:
phy = os.open("/dev/mmcblk0", os.O_RDWR)

diskSize = GetBlockDeviceSizeSafe(phy)

print("{} MiB".format(GetBlockDeviceSizeSafe(phy) / (1024 * 1024)))

stride = 16
outerStride = 4096 # Lets just do this
totalIndices = diskSize // stride
outerIndices = diskSize // outerStride

if not diskSize % stride == 0:
    print("FDE will not work here")
elif not diskSize % outerStride ==0:
    print("Hmm...")
    sys.exit(0)
    

In [ ]:
# Big Memory Mode (8MiB for both arrays)

# Prime the encryptor with a dead block
deadBlock = os.urandom(16)



# We'll use 4M block sectors
# So we need a 16b carryover

carryover = deadBlock

# While loop over remaining size
remainingDisk = diskSize

sectorSize = 4 * 1024 * 1024
blocksPerSector = sectorSize // 16


while not remainingDisk == 0:
    writeData = numpy.zeros(sectorSize, dtype=(numpy.void, 1))
    
    calculatedRead = min(remainingDisk, sectorSize)
    
    data = os.pread(phy, calculatedRead, diskSize - remainingDisk)
    
    # Check is divisable by 16
    if not calculatedRead % 16 == 0:
        print("How?")
        sys.exit(1)
    
    # Do the first block from carryover
    writeData[0:16] = numpy.frombuffer(encryptor.update(carryover), dtype=(numpy.void, 1))
    
    # Write into buffer
    writeData[16:calculatedRead] = numpy.frombuffer(encryptor.update(data[:calculatedRead - 16]), dtype=(numpy.void, 1))
    
    # Reset carryover
    carryover = data[calculatedRead - 16:]
    
    os.pwrite(phy, writeData, diskSize - remainingDisk)
    
    # Remaining
    remainingDisk -= calculatedRead
    
    # Status
    print("\r{} bytes processed. {:.2f} percent complete!".format(
            diskSize - remainingDisk,
            ((diskSize - remainingDisk) / diskSize) * 100
        ),
        end='', flush=True)
    
print("Discarded bytes: {}".format(carryover))
    

In [ ]:
## Big Memory Mode (12MiB for both arrays)

# While loop over remaining size
remainingDisk = diskSize

sectorSize = 4 * 1024 * 1024
blocksPerSector = sectorSize // 16

writeData = numpy.zeros(sectorSize * 2, dtype=(numpy.void, 1))
writeDataLength = 0

# Do first loop, it's special
calculatedRead = min(remainingDisk, sectorSize)    
data = os.pread(phy, calculatedRead, diskSize - remainingDisk)
writeData[0:calculatedRead - 16] = numpy.frombuffer(decryptor.update(data), dtype=(numpy.void, 1))[16:]
writeDataLength += (calculatedRead - 16)
remainingDisk -= calculatedRead
writtenLength = 0

while not remainingDisk == 0:
    calculatedRead = min(remainingDisk, sectorSize)
    
    data = os.pread(phy, calculatedRead, diskSize - remainingDisk)
    print("Reading from {}".format(diskSize - remainingDisk))
    
    # Check is divisable by 16
    if not calculatedRead % 16 == 0:
        print("How?")
        sys.exit(1)
     
    # Copy to buffer
    writeData[writeDataLength:writeDataLength + calculatedRead] = numpy.frombuffer(
        decryptor.update(data), dtype=(numpy.void, 1))
    writeDataLength += calculatedRead
    
    if writeDataLength >= sectorSize:
        # Write sector to disk
        # MaxDisk - remaining gives the read head
        # Remove the amount of data not yet written
        print("Writing to {}".format((diskSize - remainingDisk) - sectorSize))
        lWrite = os.pwrite(phy, writeData[0: sectorSize], (diskSize - remainingDisk) - sectorSize)
        writeDataLength -= lWrite
        writeData = numpy.roll(writeData, lWrite)
    
    # Remaining
    remainingDisk -= calculatedRead
    
    # Status
    print("\r{} bytes processed. {:.2f} percent complete!".format(
            diskSize - remainingDisk,
            ((diskSize - remainingDisk) / diskSize) * 100
        ),
        end='', flush=True)
    
# Padding
writeData[writeDataLength : sectorSize * 2] = b'' * ((sectorSize * 2) - writeDataLength)

# Eride
os.pwrite(phy, writeData[0:sectorSize], diskSize - writeDataLength)

In [ ]:
os.close(phy)

In [ ]:

# Prime the encryptor with a dead block
deadBlock = os.urandom(16)

shiftArray = [deadBlock, b""]

for i in range(0,totalIndices):
    shiftArray[1] = os.pread(phy, stride, i * stride)
    
    encData = encryptor.update(shiftArray[0])
    
    os.pwrite(phy, encData, i * stride)
    
    shiftArray[0] = shiftArray[1]
    
#encryptor.finalize()
    
    
    


In [ ]:
decryptor.update(os.pread(phy, stride, 0))

array = b""
for i in range(1, 1000):#totalIndices):
    array += decryptor.update(os.pread(phy, stride, i * stride))
    
print(array)

In [ ]:
def eta(data, unit='natural'):
    base = {
        'shannon' : 2.,
        'natural' : math.exp(1),
        'hartley' : 10.
    }

    if len(data) <= 1:
        return 0

    counts = Counter()

    for d in data:
        counts[d] += 1

    ent = 0

    probs = [float(c) / len(data) for c in counts.values()]
    for p in probs:
        if p > 0.:
            ent -= p * math.log(p, base[unit])

    return ent

In [ ]:
for j in range(0, outerIndices // 1024):
    data = os.pread(phy, outerStride * 1024, j * outerStride * 1024)
    
    for i in range(0, outerStride):
        ent = eta(data[i * outerStride:i * outerStride + outerStride - 1])
    
        if not ent == 0.0:
            print("Block {} has entropy of {}".format(j * 1024 + i, ent))